## FEATURE TRACKING

In [1]:
import cv2
import numpy as np

def init_tracker():
    # Inicializar el detector de características SIFT
    feature_detector = cv2.SIFT_create()
    
    # Inicializar el matcher de características
    matcher = cv2.BFMatcher()
    
    return feature_detector, matcher

def track_features(frame1, frame2, feature_detector, matcher):
    # Detectar keypoints y descriptores
    keypoints1, descriptors1 = feature_detector.detectAndCompute(frame1, None)
    keypoints2, descriptors2 = feature_detector.detectAndCompute(frame2, None)
    
    if descriptors1 is None or descriptors2 is None:
        return [], [], []
    
    # Encontrar matches entre los descriptores
    matches = matcher.knnMatch(descriptors1, descriptors2, k=2)
    
    # Aplicar test de ratio de Lowe para filtrar buenos matches
    good_matches = []
    for m, n in matches:
        if m.distance < 0.7 * n.distance:
            good_matches.append(m)
    
    # Obtener coordenadas de los puntos coincidentes
    if len(good_matches) > 0:
        pts1 = np.float32([keypoints1[m.queryIdx].pt for m in good_matches])
        pts2 = np.float32([keypoints2[m.trainIdx].pt for m in good_matches])
        return pts1, pts2, good_matches
    
    return [], [], []

def main():
    # Iniciar captura de video
    cap = cv2.VideoCapture(0)
    
    # Verificar si la cámara se abrió correctamente
    if not cap.isOpened():
        print("Error al abrir la cámara")
        return
    
    # Obtener el primer frame
    ret, prev_frame = cap.read()
    if not ret:
        print("Error al leer el primer frame")
        return
    
    # Convertir a escala de grises
    prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)
    
    # Inicializar el tracker
    feature_detector, matcher = init_tracker()
    
    while True:
        # Leer el siguiente frame
        ret, frame = cap.read()
        if not ret:
            break
            
        # Convertir a escala de grises
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        
        # Realizar el tracking
        pts1, pts2, good_matches = track_features(prev_gray, gray, feature_detector, matcher)
        
        # Dibujar las trayectorias
        if len(pts1) > 0:
            for pt1, pt2 in zip(pts1, pts2):
                # Dibujar una línea entre los puntos coincidentes
                cv2.line(frame, tuple(map(int, pt1)), tuple(map(int, pt2)), (0, 255, 0), 2)
                # Dibujar un círculo en el punto actual
                cv2.circle(frame, tuple(map(int, pt2)), 5, (0, 0, 255), -1)
        
        # Mostrar el resultado
        cv2.imshow('Feature Tracking', frame)
        
        # Actualizar el frame previo
        prev_gray = gray.copy()
        
        # Salir si se presiona 'q'
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    # Liberar recursos
    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()

## OPTICAL FLOW

In [2]:
import numpy as np
import cv2

def init_tracking_points():
    # Parámetros para la detección de esquinas de Shi-Tomasi
    feature_params = dict(
        maxCorners=100,
        qualityLevel=0.3,
        minDistance=7,
        blockSize=7
    )
    return feature_params

def init_optical_flow():
    # Parámetros para el flujo óptico de Lucas-Kanade
    lk_params = dict(
        winSize=(15, 15),
        maxLevel=2,
        criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03)
    )
    return lk_params

def main():
    cap = cv2.VideoCapture(0)
    
    # Verificar si la cámara se abrió correctamente
    if not cap.isOpened():
        print("Error al abrir la cámara")
        return

    # Leer el primer frame
    ret, old_frame = cap.read()
    if not ret:
        print("Error al leer el primer frame")
        return

    # Convertir a escala de grises
    old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)
    
    # Inicializar parámetros
    feature_params = init_tracking_points()
    lk_params = init_optical_flow()
    
    # Crear máscara para dibujar
    mask = np.zeros_like(old_frame)
    
    # Detectar puntos iniciales
    p0 = cv2.goodFeaturesToTrack(old_gray, mask=None, **feature_params)
    
    # Crear colores aleatorios para las trayectorias
    color = np.random.randint(0, 255, (100, 3))

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        if p0 is not None and len(p0) > 0:
            # Calcular el flujo óptico
            p1, st, err = cv2.calcOpticalFlowPyrLK(
                old_gray, 
                frame_gray, 
                p0, 
                None, 
                **lk_params
            )

            if p1 is not None:
                # Seleccionar puntos buenos
                good_new = p1[st == 1]
                good_old = p0[st == 1]

                # Dibujar las trayectorias
                for i, (new, old) in enumerate(zip(good_new, good_old)):
                    a, b = new.ravel()
                    c, d = old.ravel()
                    
                    # Dibujar línea entre punto antiguo y nuevo
                    mask = cv2.line(mask, (int(a), int(b)), (int(c), int(d)), 
                                  color[i % len(color)].tolist(), 2)
                    
                    # Dibujar círculo en la posición actual
                    frame = cv2.circle(frame, (int(a), int(b)), 5, 
                                     color[i % len(color)].tolist(), -1)
                
                # Superponer máscara y frame
                img = cv2.add(frame, mask)
                
                # Mostrar resultado
                cv2.imshow('Optical Flow', img)
                
                # Actualizar puntos anteriores
                p0 = good_new.reshape(-1, 1, 2)
            
            # Si se perdieron demasiados puntos, detectar nuevos
            if len(good_new) < 10:
                p0 = cv2.goodFeaturesToTrack(frame_gray, mask=None, **feature_params)
                mask = np.zeros_like(old_frame)

        else:
            # Si no hay puntos, detectar nuevos
            p0 = cv2.goodFeaturesToTrack(frame_gray, mask=None, **feature_params)
            mask = np.zeros_like(old_frame)

        # Actualizar frame anterior
        old_gray = frame_gray.copy()

        # Salir si se presiona 'q'
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

        # Limpiar trayectorias si se presiona 'c'
        if cv2.waitKey(1) & 0xFF == ord('c'):
            mask = np.zeros_like(old_frame)

    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()

In [4]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pywt

# Capturar video
ruta_del_video = "FLUJO OPTICO/MAN_WALK.mp4"
cap = cv2.VideoCapture(ruta_del_video)
fgbg = cv2.createBackgroundSubtractorMOG2()

# Diccionario para almacenar trayectorias
trajectories = {}

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    fgmask = fgbg.apply(gray)
    
    # Encontrar contornos
    contours, _ = cv2.findContours(fgmask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    for i, cnt in enumerate(contours):
        if cv2.contourArea(cnt) > 500:  # Filtrar pequeños ruidos
            x, y, w, h = cv2.boundingRect(cnt)
            center = (x + w // 2, y + h // 2)
            
            # Almacenar trayectorias
            if i not in trajectories:
                trajectories[i] = []
            trajectories[i].append(center)
            
            # Dibujar trayectorias
            for j in range(1, len(trajectories[i])):
                cv2.line(frame, trajectories[i][j-1], trajectories[i][j], (0, 255, 0), 2)
            
            # Dibujar el bounding box
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 0, 255), 2)
    
    cv2.imshow('Frame', frame)
    cv2.imshow('Foreground Mask', fgmask)
    
    if cv2.waitKey(30) & 0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()